# Introdução:

A partir do arquivo [Introdução](Introdução.ipynb), foi possível identificar parâmetros e características das Features e dos Targets. A partir disso, iremos treinar uma Rede Neural MLP (Multi Layer Perceptron), com o uso do Pytorch, com diversas arquiteturas distintas, através do Optuna, para identificar os melhores hiperparâmetros para a previsão dos nossos Targets.

As Redes Neurais funcionam a partir de Camadas de Neurônios, que recebem uma informação com um determinado peso e processa essa dado através de uma função de ativação e um viés, resultando num output que é passado para outro neurônio conectado ou final.
As Redes Neurais possuem numeros variáveis de camadas e neurônios em cada camada. Dessa forma, sendo hiperparâmetros da arquitetura da nossa rede.
 
Para isso, iremos usar o Optuna, que consegue calcular a métricas de diversas arquiteturas de Redes Neurais. Assim, escolhendo os hiperparâmetros que funcionam melhor para a predição dos nossos targets.

# Desenvolvimento:

## Importações e definições:

Importando as bibliotecas necessárias:

In [1]:
import pandas as pd
import numpy as np
import os
import optuna

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
import torch.optim as optim


Definindo as constantes do problema:

In [2]:
TAMANHO_TESTE = 0.1
TAMANHO_VALIDACAO = 0.1

SEMENTE_ALEATORIA = 1249

NUM_EPOCAS = 50

## Pré-processamento:

Abaixo serão descritas as etapas de pré-processamento explicadas no notebook anterior.

--- CORRIGIR, PRECISAMOS DIVIDIR O DATASET EM TREINO, VALIDAÇÃO E TESTE E DEPOIS NORMALIZAR AJUSTANDO AO TREINO E APLIANDO AO RESTO (VAMOS DESNORMALIZAR DEPOIS), MAS NÃO É BOM NORMALIZAR TODOS OS ATRIBUTOS

Carregando o dataset de energia:

In [3]:
TARGET = ["Storage_Efficiency_Percentage"]
FEATURES = [
    "Type_of_Renewable_Energy",
    "Installed_Capacity_MW",
    "Energy_Production_MWh",
    "Energy_Consumption_MWh",
    "Energy_Storage_Capacity_MWh",
    "Grid_Integration_Level",
    "Initial_Investment_USD",
    "Funding_Sources",
    "Financial_Incentives_USD",
    "GHG_Emission_Reduction_tCO2e",
    "Air_Pollution_Reduction_Index",
]

df_energia = pd.read_csv('energy_dataset_.csv')
df_energia = df_energia.reindex(FEATURES + TARGET, axis=1)
df_energia = df_energia.dropna()
df_energia

,Type_of_Renewable_Energy,Installed_Capacity_MW,Energy_Production_MWh,Energy_Consumption_MWh,Energy_Storage_Capacity_MWh,Grid_Integration_Level,Initial_Investment_USD,Funding_Sources,Financial_Incentives_USD,GHG_Emission_Reduction_tCO2e,Air_Pollution_Reduction_Index,Storage_Efficiency_Percentage
0,4,93.423205,103853.2206,248708.4892,2953.248771,4,4.732248e+08,1,9.207772e+06,6663.816572,81.742461,89.887562
1,4,590.468942,190223.0649,166104.1642,5305.174042,4,1.670697e+08,2,1.685101e+06,30656.049820,78.139042,84.403343
2,1,625.951142,266023.4824,424114.6308,2620.192622,2,8.463610e+07,2,5.111813e+06,1749.613759,8.461296,60.498249
3,1,779.998728,487039.5296,308337.7316,1925.250307,3,3.967690e+08,2,4.805902e+06,43233.237820,8.402441,86.897861
4,3,242.106837,482815.0856,360437.7705,3948.945383,2,3.574413e+07,1,1.668601e+07,14858.662760,28.822867,70.949351
...,...,...,...,...,...,...,...,...,...,...,...,...
14995,3,745.032555,280007.5738,230544.8268,4351.687893,4,3.484136e+08,2,1.558508e+07,25234.911810,78.923200,90.791405
14996,1,15.187023,377340.5803,358547.3589,6792.194696,4,2.560179e+08,3,6.866618e+06,15762.519790,54.982974,78.252040
14997,3,877.539059,480497.3920,214441.6719,4588.725297,1,1.300112e+08,2,3.837764e+06,44597.809410,43.915897,58.282928
14998,7,551.264716,436383.1694,137043.8713,7251.144215,2,3.334831e+08,2,5.347706e+06,34363.858000,4.877145,73.573666


Aplicando a normalização por Máximo Absoluto considerando as colunas fixas, que contém dados discretos e necessários para o treinamento e interpretabilidade do algoritmo:

In [12]:
# df_energia_normalizado = df_energia.copy()
# colunas_fixas = ['Type_of_Renewable_Energy', 'Grid_Integration_Level','Funding_Sources']
# colunas_variaveis = df_energia_normalizado.columns.difference(colunas_fixas)

# normalizador = MaxAbsScaler()
# df_energia_normalizado[colunas_variaveis] = normalizador.fit_transform(df_energia_normalizado[colunas_variaveis])
# df_energia_normalizado

Dividindo os dados em treino, validação e teste, permitindo que a rede neural se ajuste aos dados de treino e seja otimizada para os dados de validação, evitando o *overfitting*:

In [13]:
# Separação dos dados de teste
indices = df_energia.index
indices_treino_val, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino_val = df_energia.loc[indices_treino_val]
df_teste = df_energia.loc[indices_teste]

X_teste = df_teste.reindex(FEATURES, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values

In [6]:
# Separação dos dados de treino e validação
indices = df_treino_val.index
indices_treino, indices_val = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df_energia.loc[indices_treino]
df_val = df_energia.loc[indices_val]

X_treino = df_treino.reindex(FEATURES, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values

X_val = df_val.reindex(FEATURES, axis=1).values
y_val = df_val.reindex(TARGET, axis=1).values

Fazendo uma normalização por Máximo Absoluto, ajustando aos dados de treino e transformando aos demais dados:

In [7]:
norm_x = MaxAbsScaler()
norm_x.fit(X_treino)

norm_y = MaxAbsScaler()
norm_y.fit(y_treino)

X_treino = norm_x.transform(X_treino)
y_treino = norm_y.transform(y_treino)

X_val = norm_x.transform(X_val)
y_val = norm_y.transform(y_val)

X_teste = norm_x.transform(X_teste)
y_teste = norm_y.transform(y_teste)


Convertendo os dados para Tensores, uma estrutura especial utilizada no módulo `Pytorch`:

In [8]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)

X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

## Optuna:

In [ ]:
def define_model(trial):
    # ver questão do randomstate
    n_camadas = trial.suggest_int("n_camadas", 2, 5)
    camadas = []
 
    in_features = len(FEATURES)
    for i in range(n_camadas):
        out_features = trial.suggest_int(f"n_neuronios{i}", 2, 5)
        camadas.append(nn.Linear(in_features, out_features))
        camadas.append(nn.ReLU())
        in_features = out_features
 
    camadas.append(nn.Linear(in_features, NUM_TARGETS))

 
    return nn.Sequential(*camadas)

In [ ]:
def objective(trial):
    model = define_model(trial).to(DEVICE)
 
    nome_otimizador = trial.suggest_categorical("otimizador", ["Adam"])
    lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True) # depois ver o log
    otimizador = getattr(optim, nome_otimizador)(model.parameters(), lr=lr)
 
    treino = X_treino
    target_treino = y_treino #COLOCAR NOSSOS DADOS DE TREINO AQUI
    validacao = X_val #COLOCAR NOSSOS DADOS DE VALIDACAO AQUI
    target_validacao = y_val
 
    melhor_loss = float("inf")
    epochs_no_improve = 0
 
    print(f"Trial {trial.number}")
 
    for epoch in range(EPOCHS):        
        model.train() #modo de treinamento
 
        for data, target in zip(treino, target_treino):
            data, target = data.to(DEVICE), target.to(DEVICE)
            #Foward pass
            output = model(data)
 
            #Zero Grad
            otimizador.zero_grad()
            #Calculo da perda
            loss = F.mse_loss(output, target)
 
            #Backpropagation
            loss.backward()
            #Atualiza os parâmetros
            otimizador.step()
 
        # validação
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for data, target in zip(validacao, target_validacao):
                data, target = data.to(DEVICE), target.to(DEVICE)
                output = model(data)
                val_loss += F.mse_loss(output, target, reduction='sum').item()
 
        val_loss /= len(validacao)
        trial.report(-val_loss, epoch)  # negativo, pois queremos minimizar a loss
 
        # early stopping
        if val_loss < (melhor_loss - MIN_DELTA):
            melhor_loss = val_loss
            contador_tol = 0
 
        else:
            contador_tol += 1
            if contador_tol >= PATIENCE:
                print(f"Parada Antecipada (Early Stopping) na época {epoch}.")
                break
 

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
 
    print(f"[Trial {trial.number} Final] Melhor Val Loss: {melhor_loss:.4f}")
    return -melhor_loss  # objetivo: maximizar valor → minimizar loss

In [10]:
study = optuna.create_study(direction="minimize")

[I 2025-06-07 15:51:25,950] A new study created in memory with name: no-name-c56d5c26-cb97-49d8-ba4f-8dfa3a2328df


In [11]:
# SO RODAR QUANDO FOR NECESSÁRIO

#study.optimize(objective, n_trials=100)  # Faz 100 testes de arquitetura

#print("Melhores hiperparâmetros:", study.best_params)

## Rede Neural MLP:

## Treinamento (Talvez):

## Curva de Aprendizado (Talvez):

## Testando a Rede:

# Conclusões:

# Referências:

[1] CASSAR, DANIEL. "". Material de aula, Ano

[2] 

[3] 